In [2]:
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, CustomJS
from bokeh.layouts import column
from bokeh.models.tools import LassoSelectTool, BoxSelectTool
from bokeh.io import output_file, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6



df = pd.read_csv('Synthetic_2_classifiers.csv')

#Adding data to the data set
df['Acorrect'] = df.apply(lambda row: row['label'] == row['classifierA_predicted_label'], axis=1)
df['Bcorrect'] = df.apply(lambda row: row['label'] == row['classifierB_predicted_label'], axis=1)
df['classifier A - Correctly Predicted Dog'] = df.apply(lambda row: row['label'] == row['classifierA_predicted_label'] and row['label'] == 'dog', axis=1)
df['classifier A - Correctly Predicted Cat'] = df.apply(lambda row: row['label'] == row['classifierA_predicted_label'] and row['label'] == 'cat', axis=1)
df['classifier B - Correctly Predicted Dog'] = df.apply(lambda row: row['label'] == row['classifierB_predicted_label'] and row['label'] == 'dog', axis=1)
df['classifier B - Correctly Predicted Cat'] = df.apply(lambda row: row['label'] == row['classifierB_predicted_label'] and row['label'] == 'cat', axis=1)

df['Onecorrect'] = df.apply(lambda row: (row['label'] == row['classifierA_predicted_label']) ^ (row['label'] == row['classifierB_predicted_label']), axis=1)
df['OnecorrectA'] = df.apply(lambda row: row['Onecorrect'] and (row['classifier A - Correctly Predicted Dog'] or row['classifier A - Correctly Predicted Cat']), axis=1)
df['OnecorrectB'] = df.apply(lambda row: row['Onecorrect'] and (row['classifier B - Correctly Predicted Dog'] or row['classifier B - Correctly Predicted Cat']), axis=1)




df['Bothwrong'] = df.apply(lambda row: row['label'] != row['classifierA_predicted_label'] and row['label'] != row['classifierB_predicted_label'], axis=1)
df['Bothcorrect'] = df.apply(lambda row: row['label'] == row['classifierA_predicted_label'] and row['label'] == row['classifierB_predicted_label'], axis=1)
df['BothcorrectCat'] = df.apply(lambda row: row['label'] == 'cat' and row['classifierA_predicted_label'] == 'cat' and row['classifierB_predicted_label'] == 'cat', axis=1)
df['BothcorrectDog'] = df.apply(lambda row: row['label'] == 'dog' and row['classifierA_predicted_label'] == 'dog' and row['classifierB_predicted_label'] == 'dog', axis=1)

df

,x,y,label,classifierA_predicted_label,classifierB_predicted_label,Acorrect,Bcorrect,classifier A - Correctly Predicted Dog,classifier A - Correctly Predicted Cat,classifier B - Correctly Predicted Dog,classifier B - Correctly Predicted Cat,Onecorrect,OnecorrectA,OnecorrectB,Bothwrong,Bothcorrect,BothcorrectCat,BothcorrectDog
0,1.049671,-0.013826,dog,cat,cat,False,False,False,False,False,False,False,False,False,True,False,False,False
1,1.064769,0.152752,dog,cat,cat,False,False,False,False,False,False,False,False,False,True,False,False,False
2,0.976584,-0.022516,dog,cat,cat,False,False,False,False,False,False,False,False,False,True,False,False,False
3,1.157920,0.078090,dog,cat,cat,False,False,False,False,False,False,False,False,False,True,False,False,False
4,0.953051,0.056051,dog,cat,cat,False,False,False,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2.182917,0.595159,cat,cat,dog,True,False,False,True,False,False,True,True,False,False,False,False,False
9996,2.015847,0.437092,cat,cat,cat,True,True,False,True,False,True,False,False,False,False,True,True,False
9997,1.902314,0.609870,cat,cat,dog,True,False,False,True,False,False,True,True,False,False,False,False,False
9998,2.026503,0.560603,cat,cat,dog,True,False,False,True,False,False,True,True,False,False,False,False,False


# both with out interactivity

In [3]:
from bokeh.models import ColumnDataSource, CustomJS, LassoSelectTool, Select, HoverTool
from bokeh.layouts import column
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap
from bokeh.palettes import Category10


data = {
    "Categories": [
        "classifier A - Correctly Predicted Dog",
        "classifier B - Correctly Predicted Dog",
        "classifier A - Correctly Predicted Cat",
        "classifier B - Correctly Predicted Cat",
    ],
    "Count": [
        (df['classifier A - Correctly Predicted Dog']).sum(),
        (df['classifier B - Correctly Predicted Dog']).sum(),
        (df['classifier A - Correctly Predicted Cat']).sum(),
        (df['classifier B - Correctly Predicted Cat']).sum(),
    ],
    "Bothcorrect": [
        ((df['classifier A - Correctly Predicted Dog'] & df['Bothcorrect']).sum()),
        ((df['classifier B - Correctly Predicted Dog'] & df['Bothcorrect']).sum()),
        ((df['classifier A - Correctly Predicted Cat'] & df['Bothcorrect']).sum()),
        ((df['classifier B - Correctly Predicted Cat'] & df['Bothcorrect']).sum()),
    ],
    "Group": ["Dog", "Dog", "Cat", "Cat"],
    "TotalCount": [
        (df['label'] == 'dog').sum(),  # Total count of 'dog' label
        (df['label'] == 'dog').sum(),  # Total count of 'dog' label
        (df['label'] == 'cat').sum(),  # Total count of 'cat' label
        (df['label'] == 'cat').sum(),  # Total count of 'cat' label
    ],
}

source = ColumnDataSource(data=data)

custom_ticks_dog = ["classifier A - Correctly Predicted Dog", "classifier B - Correctly Predicted Dog"]
custom_ticks_cat = ["classifier A - Correctly Predicted Cat", "classifier B - Correctly Predicted Cat"]

# Create a separate data source for the total count bars
total_count_source = ColumnDataSource(data=dict(
    x=["classifier A - Correctly Predicted Dog", ],
    TotalCount=[(df['label'] == 'dog').sum()],
    x1=["classifier B - Correctly Predicted Cat"],
    TotalCount1=[(df['label'] == 'cat').sum()]
))

# Create a figure for the bar chart
p_bar = figure(x_range=custom_ticks_dog + ["  "] + custom_ticks_cat, height=800, width=800,
               title="Customized Vertical Bar Chart", toolbar_location=None, tools="")

# Add total count bars on top of each group with a lighter color fill
total_count_color = "red"
total_count_color1 = "blue"

p_bar.vbar(x="x", top="TotalCount", width=2.5, source=total_count_source, color=total_count_color, alpha=0.10, legend_label="Total Count")
p_bar.vbar(x="x1", top="TotalCount1", width=2.5, source=total_count_source, color=total_count_color1, alpha=0.10, legend_label="Total Count")

# Add vertical bars for 'Count' and 'Bothcorrect' for the y-axis
p_bar.vbar(x="Categories", top="Count", width=0.4, source=source,  color="#4D4D4D")
p_bar.vbar(x="Categories", top="Bothcorrect", width=0.4, source=source, color="Black", alpha=1, legend_label="Bothcorrect")

p_bar.yaxis.axis_label = "Count"

p_bar.xaxis.major_label_orientation = "vertical"

p_bar.legend.title = "Legend"
p_bar.legend.label_text_font_size = "12px"

scatter_source = ColumnDataSource(df)
scatter_plot = figure(width=1000, height=1000, title="Scatter Plot", tools=[LassoSelectTool()])

label_colors = {'dog': 'red', 'cat': 'blue'}

for label, color in label_colors.items():
    label_df = df[df['label'] == label]
    scatter_plot.scatter('x', 'y', source=label_df, size=9, color=color, legend_label=label, fill_alpha=0.8)


bothcorrect_df = df[df['Bothcorrect']]
scatter_plot.scatter('x', 'y', source=bothcorrect_df, size=5, color='black', legend_label='Both Correct', fill_alpha=0.5)

Onecorrect_df = df[df['Onecorrect']]
scatter_plot.scatter('x', 'y', source=Onecorrect_df, size=5, color='gray', legend_label='Onecorrect', fill_alpha=0.3)


bothwrong_df = df[df['Bothwrong']]
scatter_plot.scatter('x', 'y', source=bothwrong_df, size=5, color='white', legend_label='Both Wrong', fill_alpha=0.14)

scatter_plot.legend.title = 'Labels'
scatter_plot.xaxis.axis_label = 'X-axis'
scatter_plot.yaxis.axis_label = 'Y-axis'

scatter_hover = HoverTool()
scatter_hover.tooltips = [('X', '@x'), ('Y', '@y'), ('Label', '@label')]
scatter_plot.add_tools(scatter_hover)

layout = column(scatter_plot, p_bar)

show(layout)
